# Example for utilization of powerfactory-tools -- Control

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import datetime
import typing as t

from powerfactory_tools import PowerFactoryInterface
from powerfactory_tools.utils.io import to_json
from powerfactory_tools.powerfactory_types import PowerFactoryTypes as PFTypes
from powerfactory_tools.powerfactory_types import NetworkExtendedCalcType
from powerfactory_tools.powerfactory_types import CalculationCommand

In [3]:
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
GRID_NAME = "HV_8_Bus"
EXPORT_PATH = pathlib.Path("control_action_results")
PF_USER_PROFILE = ""  # specification may be necessary
PF_INI_NAME = ""  # optional specification of ini file name to switch to full version (e.g. PowerFactoryFull for file PowerFactoryFull.ini)
PYTHON_VERSION = "3.10"

In [4]:
def export_data(
    data: dict,
    export_data_name: str | None,
    export_path: pathlib.Path,
) -> None:
    """Export data to json file.

    Arguments:
        data {dict} -- data to export
        data_name {str | None} -- the chosen file name for data
        export_path {pathlib.Path} -- the directory where the exported json file is saved
    """
    timestamp = datetime.datetime.now().astimezone()  # noqa: DTZ005
    timestamp_string = timestamp.isoformat(sep="T", timespec="seconds").replace(":", "")
    if export_data_name is None:
        filename = f"{PROJECT_NAME}_{timestamp_string}.json"
    else:
        filename = f"{export_data_name}.json"

    file_path = export_path / filename
    try:
        file_path.resolve()
    except OSError as e:
        msg = f"File path {file_path} is not a valid path."
        raise FileNotFoundError(msg) from e

    to_json(data=data, file_path=file_path)

## Control using a controller instance

In [ ]:
export_data_name = "export_data"

with PowerFactoryInterface(
    project_name=PROJECT_NAME,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PYTHON_VERSION,
) as pfi:
    ## Either use interface functions, which return typed data
    # --> Get all objects from specific grid
    data = pfi.compile_powerfactory_data(GRID_NAME)
    # All nodes in grid {grid_name}, also these that are out of service
    terminals_grid = data.terminals
    # Get all scenarios
    scenarios = pfi.scenarios()

    ## Or use PF built-in functions via pfi.app
    # All nodes of active study case which are in service
    terminals_study_case = pfi.app.GetCalcRelevantObjects("*.ElmTerm", 0)
    # --> Typing is not necessary, but recommended so that the IDE code can give some code completion proposals
    terminals_study_case: t.Sequence[PFTypes.Terminal] = pfi.app.GetCalcRelevantObjects("*.ElmTerm", 0)
    # Get only active scenario
    scenario = pfi.app.GetActiveScenario()

    ## Select only terminals with nominal voltage of xx kV
    voltage_threshold = 110
    terminals_sel = []  # selected terminals
    for term in terminals_study_case:
        # nominal voltage
        u_n = term.uknom
        if u_n == voltage_threshold:
            terminals_sel.append(term)

    # Select some loads that names start with "Load"
    loads = pfi.loads(name="Load*")

    # Change power of loads
    for load in loads:
        load.plini = 2  # active power in MW

    ## Init load flow - sym or unsym
    # Typing is not necessary, but recommended so that the IDE code can give some code completion proposals
    ldf = t.cast(
        "PFTypes.CommandLoadFlow",
        pfi.app.GetFromStudyCase(CalculationCommand.LOAD_FLOW.value),
    )
    ldf.iopt_net = NetworkExtendedCalcType.AC_UNSYM_ABC.value
    error: int = ldf.Execute()
    if error != 0:
        print("Load flow execution failed.")

    result = pfi.result(name="All*")
    # Do further user specific work and fill result_data dictonairy based on PF result
    result_data = {}

    ## Store results
    export_data(
        data=result_data,
        export_data_name=export_data_name,
        export_path=EXPORT_PATH,
    )

## Control using the control function running in a new process with default parameters

In [ ]:
import multiprocessing

POWERFACTORY_PATH = pathlib.Path("C:/Program Files/DIgSILENT")
POWERFACTORY_VERSION = "2022 SP2"


class PowerFactoryControllerProcess(multiprocessing.Process):
    def __init__(
        self,
        *,
        export_path: pathlib.Path,
        project_name: str,
        grid_name: str,
        export_data_name: str = "",
        powerfactory_user_profile: str = "",
        powerfactory_path: pathlib.Path = POWERFACTORY_PATH,
        powerfactory_version: str = POWERFACTORY_VERSION,
        python_version: str = PYTHON_VERSION,
    ) -> None:
        super().__init__()
        self.export_path = export_path
        self.project_name = project_name
        self.grid_name = grid_name
        self.export_data_name = export_data_name
        self.powerfactory_user_profile = powerfactory_user_profile
        self.powerfactory_path = powerfactory_path
        self.powerfactory_version = powerfactory_version
        self.python_version = python_version

    def run(self) -> None:
        with PowerFactoryInterface(
            project_name=self.project_name,
            powerfactory_user_profile=self.powerfactory_user_profile,
            powerfactory_path=self.powerfactory_path,
            powerfactory_version=self.powerfactory_version,
            python_version=self.python_version,
        ) as pfi:
            ## Either use interface functions, which return typed data
            # --> Get all objects from specific grid
            data = pfi.compile_powerfactory_data(GRID_NAME)
            # All nodes in grid {grid_name}, also these that are out of service
            terminals_grid = data.terminals
            # Get all scenarios
            scenarios = pfi.scenarios()

            ## Or use PF built-in functions via pfi.app
            # All nodes of active study case which are in service
            terminals_study_case = pfi.app.GetCalcRelevantObjects("*.ElmTerm", 0)
            # --> Typing is not necessary, but recommended so that the IDE code can give some code completion proposals
            terminals_study_case: t.Sequence[PFTypes.Terminal] = pfi.app.GetCalcRelevantObjects("*.ElmTerm", 0)
            # Get only active scenario
            scenario = pfi.app.GetActiveScenario()

            ## Select only terminals with nominal voltage of xx kV
            voltage_threshold = 110
            terminals_sel = []  # selected terminals
            for term in terminals_study_case:
                # nominal voltage
                u_n = term.uknom
                if u_n == voltage_threshold:
                    terminals_sel.append(term)

            # Select some loads that names start with "Load"
            loads = pfi.loads(name="Load*")

            # Change power of loads
            for load in loads:
                load.plini = 2  # active power in MW

            ## Init load flow - sym or unsym
            # Typing is not necessary, but recommended so that the IDE code can give some code completion proposals
            ldf = t.cast(
                "PFTypes.CommandLoadFlow",
                pfi.app.GetFromStudyCase(CalculationCommand.LOAD_FLOW.value),
            )
            ldf.iopt_net = NetworkExtendedCalcType.AC_UNSYM_ABC.value
            error: int = ldf.Execute()
            if error != 0:
                print("Load flow execution failed.")

            result = pfi.result(name="All*")
            # Do further user specific work and fill result_data dictonairy based on PF result
            result_data = {}

            ## Store results
            export_data(
                data=result_data,
                export_data_name=export_data_name,
                export_path=EXPORT_PATH,
            )

In [ ]:
def apply_control(  # noqa: PLR0913
    export_path: pathlib.Path,
    project_name: str,
    grid_name: str,
    export_data_name: str = "",
    powerfactory_user_profile: str = "",
    powerfactory_user_password: str = "",
    powerfactory_path: pathlib.Path = POWERFACTORY_PATH,
    powerfactory_version: str = POWERFACTORY_VERSION,
    powerfactory_ini_name: str = "",
    python_version: str = PYTHON_VERSION,
) -> None:
    process = PowerFactoryControllerProcess(
        project_name=project_name,
        export_path=export_path,
        grid_name=grid_name,
        export_data_name=export_data_name,
        powerfactory_user_profile=powerfactory_user_profile,
        powerfactory_user_password=powerfactory_user_password,
        powerfactory_path=powerfactory_path,
        powerfactory_version=powerfactory_version,
        powerfactory_ini_name=powerfactory_ini_name,
        python_version=python_version,
    )
    process.start()
    process.join()

### As the control function is executed in a process that is terminated after execution, the PowerFactory API is also closed.

In [ ]:
apply_control(
    export_path=EXPORT_PATH,
    project_name=PROJECT_NAME,
    grid_name=GRID_NAME,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PYTHON_VERSION,
)